<a href="https://colab.research.google.com/github/malongfei1993/chatglm3-6b-qlora-finetuning/blob/main/chatglm3_peft_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 微调后的chatglm-6b和原始模型进行对比


*   加载微调前后的模型
*   展示微调前后模型的输入输出




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 33.5 MB/s eta 0:00:00


In [3]:
!pip install sentencepiece
!pip install bitsandbytes
!pip install accelerate
!pip install peft
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [4]:
# !pip uninstall transformers

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.35.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.35.2


这里需要指定下transformers版本

In [5]:
# !pip install transformers==4.34.0

y
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 97.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.17.0 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.17.3 which is incompatible.


In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [21]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [22]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [5]:

from peft import PeftModel, PeftConfig

epochs = 4
peft_model_path = f"drive/MyDrive/fineTuning0201/models/THUDM/chatglm3-6b-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)
# model = AutoModel.from_pretrained(peft_model_path,device_map="auto",trust_remote_code=True)

In [6]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=4608, bias=False)

In [8]:
test_text = '''

你是谁

'''

In [9]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query,temperature=0.1,history=[])

    # inputs = tokenizer(query, return_tensors="pt").to(0)
    # ft_out = model.generate(**inputs, max_new_tokens=512,temperature=0.1)
    # ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    ft_response, history = model.chat(tokenizer,query,temperature=0.1,history=[])
    # print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    print(f"原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response,ft_response

In [18]:
compare_chatglm_results(test_text)

原始输出：
我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。


ChatGLM3-6B微调后：
我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。


('我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。',
 '我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。')

In [ ]:
import pandas as pd
pd_read=pd.read_csv('drive/MyDrive/DSData/ds_data.csv')

In [ ]:
np_arr=pd_read['content']

In [ ]:
np_new_arr = np_arr[:10]

In [14]:
np_new_arr

0    \n这是数据：\n {\n        "No.": 13,\n        "Gene...
1    \n这是数据：\n{\n        "No.": 22,\n        "Gener...
2    \n这是数据：\n{\n        "No.": 22,\n        "Gener...
3    \n这是数据：\n {\n        "No.": 33,\n        "Gene...
4    \n这是数据：\n{\n        "No.": 40,\n        "Gener...
5    \n这是数据：\n {\n        "No.": 28,\n        "Gene...
6    \n这是数据：\n {\n        "No.": 29,\n        "Gene...
7    \n这是数据：\n {\n        "No.": 46,\n        "Gene...
8    \n这是数据：\n {\n        "No.": 39,\n        "Gene...
9    \n这是数据：\n {\n        "No.": 39,\n        "Gene...
Name: content, dtype: object

In [24]:
for item in np_new_arr:
  compare_chatglm_results(item)

原始输出：
[{
"参数": "Screen size min. (diagonal)",
"需求规格": "12.1",
"供应商反馈": "OK"
}]


ChatGLM3-6B微调后：
{
"参数": "diagonal size",
"需求规格": "12.1",
"供应商反馈": "OK"
}
原始输出：
[{
"参数":"horizontal resolution",
"需求规格":"1280 x 800",
"供应商反馈": "OK"
}]


ChatGLM3-6B微调后：
{
"参数": "horizontal resolution",
"需求规格": "1280",
"供应商反馈": "OK"
}
原始输出：
[{
"参数": "frame rate",
"需求规格": "It should be 50/60 Hz",
"供应商反馈": "It should be 60HZ"
}]


ChatGLM3-6B微调后：
{
"参数": "frame rate",
"需求规格": "50/60 Hz",
"供应商反馈": "60HZ"
}


KeyboardInterrupt: 